<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importando-Bibliotecas" data-toc-modified-id="Importando-Bibliotecas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importando Bibliotecas</a></span></li><li><span><a href="#Definindo-Funções" data-toc-modified-id="Definindo-Funções-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Definindo Funções</a></span></li><li><span><a href="#Lendo-os-Dados" data-toc-modified-id="Lendo-os-Dados-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Lendo os Dados</a></span></li><li><span><a href="#DataPrep" data-toc-modified-id="DataPrep-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>DataPrep</a></span></li><li><span><a href="#Exploração-Gráfica" data-toc-modified-id="Exploração-Gráfica-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Exploração Gráfica</a></span></li></ul></div>

Este notebook tem como objetivo propor uma análise da evolução do COVID-19 ao redor do mundo, enfatizando casos confirmados, vítimas e casos recuperados no `Brasil`.
Utilizando uma [base de dados](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset) do Kaggle com atualização diária, serão apresentados gráficos e análises sobre o impacto do Corona Virus na sociedade como um todo.

# Importando Bibliotecas

In [1]:
# Biblliotecas utilizadas no projeto
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')
import urllib.request, json

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

# Definindo Funções

In [2]:
# Formatando eixos do matplotlib
def format_spines(ax, right_border=True):
    """
    This function sets up borders from an axis and personalize colors
    
    Input:
        Axis and a flag for deciding or not to plot the right border
    Returns:
        Plot configuration
    """    
    # Setting up colors
    ax.spines['bottom'].set_color('#CCCCCC')
    ax.spines['left'].set_color('#CCCCCC')
    ax.spines['top'].set_visible(False)
    if right_border:
        ax.spines['right'].set_color('#CCCCCC')
    else:
        ax.spines['right'].set_color('#FFFFFF')
    ax.patch.set_facecolor('#FFFFFF')

# Lendo os Dados

In [3]:
# Lendo a base mais recente sobre o virus
data_path = r'D:\Users\thiagoPanini\github_files\kaggle_challenges\kernels\08_corona_virus\data'
df_corona = pd.read_csv(data_path + f'\covid_19_data.csv')
df_corona.columns = [c.lower().replace(' ', '_').replace('/', '_') for c in df_corona.columns]
df_corona.head()

,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


De acordo com a documentação da base disponibilizada no Kaggle, as colunas presentes na base são:

- **sno:** serial number;
- **observationdate:** data da observação no formato MM/DD/YYYY;
- **province_state:** cidade referente ao registro (pode ser vazio ("") quando nulo);
- **country_region:** país referente ao registro;
- **last_update:** tempo (UTC) na qual o registro foi atualizado para o estado e país em questão;
- **confirmed**: quantidade cumulativa de casos confirmados até a data em questão;
- **deaths:** quantidade cumulativa de vítimas até a data em questão;
- **recovered:** quantidade cumulativa de pacientes recuperados até a data em questão

In [4]:
# Volumetria
df_corona.shape

(6722, 8)

# DataPrep

Aqui será proposto um fluxo de preparação da base lida envolvendo:

**1)** Transformação das colunas de data e eliminação das inconsistências;

**2)** Criação de nova coluna com casos `ativos`

In [5]:
# Limpando colunas de data
df_corona['last_update_cleaned'] = pd.to_datetime(df_corona['last_update']).dt.date
df_corona['obs_date_cleaned'] = pd.to_datetime(df_corona['observationdate']).dt.date
df_corona.drop(['last_update', 'observationdate'], axis=1, inplace=True)
df_corona.columns = ['sno', 'province_state', 'country_region', 'confirmed', 
                     'deaths', 'recovered', 'observation_date', 'last_update']

# Criação de coluna com casos ativos
df_corona['actives'] = df_corona['confirmed'] - df_corona['deaths'] - df_corona['recovered']

df_corona.head()

,sno,province_state,country_region,confirmed,deaths,recovered,observation_date,last_update,actives
0,1,Anhui,Mainland China,1.0,0.0,0.0,2020-01-22,2020-01-22,1.0
1,2,Beijing,Mainland China,14.0,0.0,0.0,2020-01-22,2020-01-22,14.0
2,3,Chongqing,Mainland China,6.0,0.0,0.0,2020-01-22,2020-01-22,6.0
3,4,Fujian,Mainland China,1.0,0.0,0.0,2020-01-22,2020-01-22,1.0
4,5,Gansu,Mainland China,0.0,0.0,0.0,2020-01-22,2020-01-22,0.0


In [6]:
# Range dos dados
print(f'Range de observação: de {df_corona["observation_date"].min()} até {df_corona["observation_date"].max()}\n')
print(f'Range de atualização: de {df_corona["last_update"].min()} até {df_corona["last_update"].max()}')

Range de observação: de 2020-01-22 até 2020-03-18

Range de atualização: de 2020-01-22 até 2020-03-18


# Exploração Gráfica

Nesta sessão, vamos iniciar as análises gráficas a partir de plotagens com `matplotlib`, `seaborn` e `plotly`. O intuito é retirar insights da base de dados e proporcionar uma visão ampla sobre os impactos do COVID-19.

In [7]:
# Agrupando dados
corona_sum = df_corona.groupby(by='last_update', as_index=False).sum()
china_sum = df_corona.query('country_region == "Mainland China"').groupby(by='last_update', as_index=False).sum()

In [8]:
# Evolução COVID-19 no mundo
fig = go.Figure()

# Criando linha - Casos confirmados ao longo do mundo
trace0 = go.Scatter(
    x=corona_sum['last_update'],
    y=corona_sum['confirmed'],
    line=dict(
        color='black',
        width=4
    ),
    name='Mundo',
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Data</b>: %{x}<br>'
)

# Criando linha - Casos confirmados na China
trace1 = go.Scatter(
    x=china_sum['last_update'],
    y=china_sum['confirmed'],
    line=dict(
        color='crimson',
        width=2
    ),
    name='China',
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Data</b>: %{x}<br>'
)

# Criando ponto específico
trace2 = go.Scatter(
    x=corona_sum['last_update'][-1:],
    y=corona_sum['confirmed'][-1:],
    mode='markers',
    marker=dict(
        color='black',
        size=12
    ),
    showlegend=False,
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Data</b>: %{x}<br>'
)

# Adicionando traços às figuras
fig.add_trace(trace0)
fig.add_trace(trace1)
fig.add_trace(trace2)

# Formatando layout da plotagem
fig.update_layout(
    
    # Eixo x
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    
    # Eixo y
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),  
    showlegend=True,
    plot_bgcolor='white',
    
    # Título do gráfico
    title=dict(
        text='Evolução COVID-19 no Mundo',
        font=dict(
            family='Franklin Gothic',
            size=25,
            color='dimgrey'
        )
    ),
    
    # Título do eixo y
    yaxis_title=dict(
        text='Casos Confirmados',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(150,150,150)'
        )
    )
)

# Criando anotações
annotations = []
annotations.append(
    dict(
        xref='paper', 
        yref='paper', 
        x=0.5, 
        y=-0.1,
        xanchor='center', 
        yanchor='top',
        text='Fonte: <a href=https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset>' +
                                   'Novel COVID-19 Kaggle dataset</a>',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(150,150,150)'
        ),
        showarrow=False
    )
)

annotations.append(
    dict(
        x=corona_sum['last_update'][31],
        y=corona_sum['confirmed'][31],
        xref='x',
        yref='y',
        text='Espalhamento drástico <br>pelo mundo',
        ax=-20,
        ay=-50,
        showarrow=True,
        arrowhead=2,
        font=dict(
            color='dimgrey'
        )
    )
)

annotations.append(
    dict(
        x=corona_sum['last_update'][len(corona_sum)-1],
        y=corona_sum['confirmed'][len(corona_sum)-1],
        xref='x',
        yref='y',
        text=f'{int(corona_sum["confirmed"].max())}' + 
                '<br>casos confirmados<br> ao todo',
        ax=-90,
        ay=0,
        showarrow=True,
        arrowhead=2,
        font=dict(
            family='Raleway, sans-serif',
            color='dimgrey'
        )
    )
)
 
fig.update_layout(annotations=annotations)
fig.show()

In [9]:
# Agrupando dados por país e data de atualização
country_sum = df_corona.groupby(by=['country_region', 'last_update'], as_index=False).sum().query('confirmed > 0')

# Criando dataset com país e primeira data de atualização
min_dt_obs = country_sum.groupby(by='country_region', as_index=False).min().loc[:, ['country_region', 'last_update']]
min_dt_obs.columns = ['country_region', 'min_dt_update']

# Unindo os dois datasets
corona_min_dt = country_sum.merge(min_dt_obs, on='country_region', how='left')

# Calculando coluna com dias após primeira obs
corona_min_dt['days_pos_1st_obs'] = corona_min_dt['last_update'] - corona_min_dt['min_dt_update']
corona_min_dt['days_pos_1st_obs'] = corona_min_dt['days_pos_1st_obs'].dt.days

# Exemplo (China)
corona_min_dt.query('country_region == "Mainland China"').head()

,country_region,last_update,sno,confirmed,deaths,recovered,actives,min_dt_update,days_pos_1st_obs
1657,Mainland China,2020-01-22,535,547.0,17.0,28.0,502.0,2020-01-22,0
1658,Mainland China,2020-01-23,1713,639.0,18.0,30.0,591.0,2020-01-22,1
1659,Mainland China,2020-01-24,3004,916.0,26.0,36.0,854.0,2020-01-22,2
1660,Mainland China,2020-01-25,4224,1399.0,42.0,39.0,1318.0,2020-01-22,3
1661,Mainland China,2020-01-26,5548,2062.0,56.0,49.0,1957.0,2020-01-22,4


In [10]:
# Filtrando dados
brazil_data = corona_min_dt.query('country_region == "Brazil"')
italy_data = corona_min_dt.query('country_region == "Italy"')
usa_data = corona_min_dt.query('country_region == "US"')

# Parâmetros para visualização
min_days = min([df['days_pos_1st_obs'].max() for df in [brazil_data, italy_data, usa_data]])
offset = 10
filter_days = min_days + offset

# Visualizando crescimento de casos confirmados após primeira obs
fig = go.Figure()

# Criando traces - Brasil
trace_brazil = go.Scatter(
    x=brazil_data['days_pos_1st_obs'],
    y=brazil_data['confirmed'],
    name='Brasil',
    line=dict(
        color='green',
        width=4
    ),
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Dias</b>: %{x}<br>'
)
trace_brazil2 = go.Scatter(
    x=brazil_data['days_pos_1st_obs'],
    y=brazil_data['actives'],
    name='Ativos BRA',
    line=dict(
        color='mediumspringgreen',
        width=2,
        dash='dash'
    ),
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Dias</b>: %{x}<br>'
)
trace_brazil3 = go.Scatter(
    x=brazil_data['days_pos_1st_obs'][-1:],
    y=brazil_data['confirmed'][-1:],
    mode='markers',
    name='Brasil',
    marker=dict(
        color='green',
        size=12
    ),
    showlegend=False,
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Data</b>: %{x}<br>'
)

# Criando traces - Itália
trace_italy = go.Scatter(
    x=italy_data['days_pos_1st_obs'][:filter_days],
    y=italy_data['confirmed'][:filter_days],
    name='Itália',
    line=dict(
        color='cornflowerblue',
        width=4
    ),
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Dias</b>: %{x}<br>'
)
trace_italy2 = go.Scatter(
    x=italy_data['days_pos_1st_obs'][:filter_days],
    y=italy_data['actives'][:filter_days],
    name='Ativos ITA',
    line=dict(
        color='royalblue',
        width=2,
        dash='dash'
    ),
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Dias</b>: %{x}<br>'
)
trace_italy3 = go.Scatter(
    x=italy_data['days_pos_1st_obs'][:filter_days][-1:],
    y=italy_data['confirmed'][:filter_days][-1:],
    mode='markers',
    name='Itália',
    marker=dict(
        color='cornflowerblue',
        size=12
    ),
    showlegend=False,
    hovertemplate =
    '<b>Casos</b>: %{y}'+
    '<br><b>Data</b>: %{x}<br>'
)

# Adicionando linhas
fig.add_trace(trace_brazil)
fig.add_trace(trace_brazil2)
fig.add_trace(trace_brazil3)
fig.add_trace(trace_italy)
fig.add_trace(trace_italy2)
fig.add_trace(trace_italy3)

# Formatando layout da plotagem
fig.update_layout(
    
    # Eixo x
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    
    # Eixo y
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    
    
    showlegend=True,
    plot_bgcolor='white',
    
    # Título do gráfico
    title=dict(
        text='Comparação Brasil e Itália após 1ª Confirmação de COVID-19',
        font=dict(
            family='Franklin Gothic',
            size=25,
            color='dimgrey'
        )
    ),
    
    # Título do eixo y
    yaxis_title=dict(
        text='Casos Confirmados',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(150,150,150)'
        )
    ),
    
    # Título do eixo x
    xaxis_title=dict(
        text='Dias Após 1ª Confirmação',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(150,150,150)'
        )
    )
)

# Criando anotações
annotations = []
ref_data = df_corona['last_update'].max()

# Data de referência
annotations.append(
    dict(
        xref='paper', 
        yref='paper', 
        x=-0.04, 
        y=1.12,
        xanchor='left', 
        yanchor='top',
        text=f'Última atualização: {ref_data.strftime("%d/%m/%Y")}',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(150,150,150)'
        ),
        showarrow=False
    )
)

# Fonte dos dados
annotations.append(
    dict(
        xref='paper', 
        yref='paper', 
        x=0.5, 
        y=-0.16,
        xanchor='center', 
        yanchor='top',
        text='Fonte: <a href=https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset>' +
                                   'Novel COVID-19 Kaggle dataset</a>',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='rgb(150,150,150)'
        ),
        showarrow=False
    )
)

# Últimas informações sobre o Brasil
br_ult_dia = int(brazil_data['days_pos_1st_obs'][-1:].values[0])
br_ult_conf = int(brazil_data['confirmed'][-1:].values[0])
br_ult_ativ = int(brazil_data["actives"][-1:].values[0])
annotations.append(
    dict(
        xref='x', 
        yref='y', 
        x=br_ult_dia,
        y=br_ult_conf+300,
        xanchor='right', 
        yanchor='top',
        text=f'<b>Brasil:</b><br>' + 
             f'{br_ult_conf} confirmados <br>' + 
             f'{br_ult_ativ} ativos após {br_ult_dia} dias',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='green'
        ),
        showarrow=False
    )
)

# Últimas informações sobre o Brasil
ita_ult_dia = int(italy_data['days_pos_1st_obs'][:filter_days][-1:].values[0])
ita_ult_conf = int(italy_data['confirmed'][:filter_days][-1:].values[0])
ita_ult_ativ = int(italy_data["actives"][:filter_days][-1:].values[0])
annotations.append(
    dict(
        xref='x', 
        yref='y', 
        x=ita_ult_dia,
        y=ita_ult_conf+300,
        xanchor='right', 
        yanchor='top',
        text=f'<b>Itália:</b><br>' + 
             f'{ita_ult_conf} confirmados <br>' + 
             f'{ita_ult_ativ} ativos após {ita_ult_dia} dias',
        font=dict(
            family='Raleway, sans-serif',
            size=12,
            color='royalblue'
        ),
        showarrow=False
    )
)
 
fig.update_layout(annotations=annotations)
fig.show()